# Post Process Lidar

The lidar data is currently stored in pickled geopandas objects.  This is bad for future proofing and makes the datasets unusful for ther system and software.

This notebook is going to change the formatting and reupload the files in a better format!  Because seriously Im not paying another $50 in processing just to change the formating

Processed files are stored at s3://requesterpays.garyscorner.net/datasets/NewOrleansElevation/

In [1]:
import geopandas as gpd
import os
from datetime import datetime
import pickle

In [2]:
def lp(v):
    print(f"[{datetime.now()}] {v}")

In [3]:
path = f"output{os.sep}"
lp(f"Output path {path}")

[2025-03-27 13:17:01.432134] Output path output/


In [4]:
files = [f"{path}{f}" for f in os.listdir(path) if f[:18] == "AggregateLidarData" and f[-7:] == '.pickle']
lp(f"Files:  {files}")

[2025-03-27 13:17:01.437278] Files:  ['output/AggregateLidarData_1000m_Previous.pickle', 'output/AggregateLidarData_SomeHoods_10m.pickle', 'output/AggregateLidarData_100m.pickle', 'output/AggregateLidarData_2000m.pickle', 'output/AggregateLidarData_1000m.pickle', 'output/AggregateLidarData_SomeHoods_1000m.pickle']


In [5]:
newFiles = []
for file in files:
    newFileName = file[:-7] + '.parquet'
    newFiles.append(newFileName)
    lp(f"processing {file} -> {newFileName}...")

    lp(f"Loading {file}")
    with open(file, 'rb') as f:
        data = pickle.load(f)

    lp(f"Saving to {newFileName}")
    data.to_parquet(newFileName)
    

[2025-03-27 13:17:01.442099] processing output/AggregateLidarData_1000m_Previous.pickle -> output/AggregateLidarData_1000m_Previous.parquet...
[2025-03-27 13:17:01.442168] Loading output/AggregateLidarData_1000m_Previous.pickle
[2025-03-27 13:17:01.444948] Saving to output/AggregateLidarData_1000m_Previous.parquet
[2025-03-27 13:17:01.472922] processing output/AggregateLidarData_SomeHoods_10m.pickle -> output/AggregateLidarData_SomeHoods_10m.parquet...
[2025-03-27 13:17:01.472996] Loading output/AggregateLidarData_SomeHoods_10m.pickle


/tmp/ipykernel_338619/2632969751.py:9: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.load(f)
/tmp/ipykernel_338619/2632969751.py:9: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, u

[2025-03-27 13:17:01.803970] Saving to output/AggregateLidarData_SomeHoods_10m.parquet
[2025-03-27 13:17:02.178307] processing output/AggregateLidarData_100m.pickle -> output/AggregateLidarData_100m.parquet...
[2025-03-27 13:17:02.178408] Loading output/AggregateLidarData_100m.pickle
[2025-03-27 13:17:02.240202] Saving to output/AggregateLidarData_100m.parquet
[2025-03-27 13:17:02.339876] processing output/AggregateLidarData_2000m.pickle -> output/AggregateLidarData_2000m.parquet...
[2025-03-27 13:17:02.339933] Loading output/AggregateLidarData_2000m.pickle
[2025-03-27 13:17:02.351990] Saving to output/AggregateLidarData_2000m.parquet
[2025-03-27 13:17:02.355503] processing output/AggregateLidarData_1000m.pickle -> output/AggregateLidarData_1000m.parquet...
[2025-03-27 13:17:02.355544] Loading output/AggregateLidarData_1000m.pickle
[2025-03-27 13:17:02.356360] Saving to output/AggregateLidarData_1000m.parquet
[2025-03-27 13:17:02.360790] processing output/AggregateLidarData_SomeHoods_1

/tmp/ipykernel_338619/2632969751.py:9: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  data = pickle.load(f)
/tmp/ipykernel_338619/2632969751.py:9: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, u

In [6]:
for file in newFiles:
    !aws s3 cp $file  s3://requesterpays.garyscorner.net/datasets/NewOrleansElevation/output/ --include="*.parquet"

upload: output/AggregateLidarData_1000m_Previous.parquet to s3://requesterpays.garyscorner.net/datasets/NewOrleansElevation/output/AggregateLidarData_1000m_Previous.parquet
upload: output/AggregateLidarData_SomeHoods_10m.parquet to s3://requesterpays.garyscorner.net/datasets/NewOrleansElevation/output/AggregateLidarData_SomeHoods_10m.parquet
upload: output/AggregateLidarData_100m.parquet to s3://requesterpays.garyscorner.net/datasets/NewOrleansElevation/output/AggregateLidarData_100m.parquet
upload: output/AggregateLidarData_2000m.parquet to s3://requesterpays.garyscorner.net/datasets/NewOrleansElevation/output/AggregateLidarData_2000m.parquet
upload: output/AggregateLidarData_1000m.parquet to s3://requesterpays.garyscorner.net/datasets/NewOrleansElevation/output/AggregateLidarData_1000m.parquet
upload: output/AggregateLidarData_SomeHoods_1000m.parquet to s3://requesterpays.garyscorner.net/datasets/NewOrleansElevation/output/AggregateLidarData_SomeHoods_1000m.parquet


In [7]:
lp("Done")

[2025-03-27 13:17:17.794242] Done
